In [1]:
import arcpy
from datetime import date
import timeit
import pandas as pd
from itertools import groupby
from operator import itemgetter

In [2]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection, FeatureLayer

In [3]:
gis = GIS('home')

In [3]:
search_result = gis.content.search(query="title:FERNSSummary_McKenzie_Joined", item_type="Feature Layer")

In [4]:
item = gis.content.get('5a6cd762152a48cc9be95b9ccf6272bd')

In [5]:
lyr = [lyr.url for lyr in item.layers if lyr.properties.name == 'FERNSSummary_McKenzie_Joined'][0]
print(lyr)

https://services5.arcgis.com/9s1YtFmLS0YTl10F/arcgis/rest/services/FERNS_for_McKenzie_Catchments/FeatureServer/0


In [63]:
lay = item.layers[0]

In [69]:
sdf = pd.DataFrame.spatial.from_layer(lay)

In [65]:
year

'2023'

In [68]:
la_df = lay.delete_features(where = f"YEAR = {year}")

In [70]:
sdf[sdf.YEAR == '2023'].shape

(0, 13)

In [57]:
sdf_old = sdf[sdf.YEAR != '2023']

In [14]:
path = r"G:\projects\UtilityDistricts\eweb\DrinkingWater\EPA319_NPS_grant\ForestApplication\ODF_FACTS_DB\FERNS\FERNS_Model"
wrkspace = path + "\\FERNS_Workspace.gdb"
year = str(date.today().year)

In [15]:
Expression="YEAR <> '{0}'".format(year)

In [32]:
arcpy.env.workspace = wrkspace

In [16]:
arcpy.env.overwriteOutput = True

In [17]:
FERNSSummary_McKenzie_Joined = path + "\\FERNS_Final_Products.gdb\\FERNSSummary_McKenzie_Joined"
res = arcpy.GetCount_management(FERNSSummary_McKenzie_Joined)

In [71]:
AGO_FERNSSummary_McKenzie_Joined_3_ = "https://services5.arcgis.com/9s1YtFmLS0YTl10F/arcgis/rest/services/FERNS_for_McKenzie_Catchments/FeatureServer/0"

In [55]:
AGO_FERNSSummary_McKenzie_Joined_5_, count = arcpy.management.SelectLayerByAttribute(in_layer_or_view=AGO_FERNSSummary_McKenzie_Joined_3_, 
                                                                                        selection_type="NEW_SELECTION", 
                                                                                        where_clause=Expression, 
                                                                                        invert_where_clause="")

In [30]:
print(f"Before Year {year} there are {count} counts and in Year {year} there are {res[0]} counts!")

Before Year 2023 there are 1351 counts and in Year 2023 there are 60 counts!


In [56]:
arcpy.management.CopyFeatures(AGO_FERNSSummary_McKenzie_Joined_5_, f'FERNS_before_{year}')

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\EPA319_NPS_grant\\ForestApplication\\ODF_FACTS_DB\\FERNS\\FERNS_Model\\FERNS_Workspace.gdb\\FERNS_before_2023'>

In [72]:
AGO_FERNSSummary_McKenzie_Joined = arcpy.management.Append(inputs=[FERNSSummary_McKenzie_Joined], 
                                                           target=AGO_FERNSSummary_McKenzie_Joined_3_, 
                                                           schema_type="NO_TEST", 
                                                           field_mapping="", 
                                                           subtype="", expression="")[0]

In [26]:
arcpy.GetCount_management(FERNSSummary_McKenzie_Joined)

<Result '60'>

In [73]:
arcpy.GetCount_management(AGO_FERNSSummary_McKenzie_Joined)

<Result '1411'>

In [59]:
sdf_new = pd.DataFrame.spatial.from_featureclass(FERNSSummary_McKenzie_Joined)

In [60]:
sdf_new.head(3)

,OBJECTID1,Basin_Name,EWEB_ID,Subbasin_Acres,YEAR,ActType,FREQUENCY,SUM_calc_acres,Activity_Acres_Percentage,SHAPE
0,1,McKenzie_E060,E060,622.140939,2023,Commercial Thinning/Selective Cutting,5,206.026276,33.115692,"{""rings"": [[[4340055.160761148, 891324.6981627..."
1,2,Holden Creek,E950,2765.264989,2023,Commercial Thinning/Selective Cutting,1,15.068833,0.544933,"{""rings"": [[[4336708.162729666, 911833.2677165..."
2,3,McKenzie/Honey Creek,E101,803.599805,2023,Herbicide Application (Unit),1,223.382093,27.797679,"{""rings"": [[[4374837.695538059, 911603.5721784..."


In [61]:
updated_sdf = sdf_new.append(sdf_old, ignore_index=False)

In [62]:
updated_sdf.shape

(1411, 13)

In [ ]:
.delete_features(where = "OBJECTID > 15")

In [10]:
import arcpy
import os
from datetime import date
from sys import argv
import timeit

In [6]:
path = r"G:\projects\UtilityDistricts\eweb\DrinkingWater\EPA319_NPS_grant\ForestApplication\ODF_FACTS_DB\FERNS\FERNS_Model"
wrkspace = path + "\\FERNS_Workspace.gdb"

In [7]:
year = str(date.today().year)

In [10]:
desc = arcpy.Describe(lyr)

In [11]:
desc

catalogPath,https://services5.arcgis.com/9s1YtFmLS0YTl10F/arcgis/rest/services/FERNS_for_McKenzie_Catchments/FeatureServer/0
dataType,FeatureClass
shapeType,Polygon
hasM,False
hasZ,False
OBJECTID1,OID
Basin_Name,String
EWEB_ID,String
Subbasin_Acres,Double
YEAR,String
ActType,String


In [7]:
def unique_values(table, field):  ##uses list comprehension
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

In [13]:
try:
    start = timeit.default_timer()
    vals = unique_values(lyr,"YEAR")

    stop = timeit.default_timer()
    total_time = stop - start
    print(total_time)
    print(vals)

except Exception as e:
    print("Error: " + e.args[0])

0.8151014000000032
['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']


In [38]:
fields = arcpy.ListFields(lyr)

In [39]:
for field in fields:
    print(f"{field.name} is a type of {field.type} with a length of {field.length}")

OBJECTID1 is a type of OID with a length of 0
Basin_Name is a type of String with a length of 10485758
EWEB_ID is a type of String with a length of 10485758
Subbasin_Acres is a type of Double with a length of 0
YEAR is a type of String with a length of 10485758
ActType is a type of String with a length of 10485758
FREQUENCY is a type of Integer with a length of 0
SUM_calc_acres is a type of Double with a length of 0
Activity_Acres_Percentage is a type of Double with a length of 0
Shape__Area is a type of Double with a length of 0
Shape__Length is a type of Double with a length of 0
GlobalID is a type of GlobalID with a length of 38
Shape is a type of Geometry with a length of 0


In [40]:
field_names = [field.name for field in fields]

In [41]:
field_names.remove('OBJECTID1')

In [42]:
field_names.remove('GlobalID')

In [43]:
field_names

['Basin_Name',
 'EWEB_ID',
 'Subbasin_Acres',
 'YEAR',
 'ActType',
 'FREQUENCY',
 'SUM_calc_acres',
 'Activity_Acres_Percentage',
 'Shape__Area',
 'Shape__Length',
 'Shape']

In [45]:
Expression="YEAR = '{0}'".format(year)
AGO_FERNSSummary_McKenzie_Joined_2_, Count = arcpy.management.SelectLayerByAttribute(in_layer_or_view=lyr, 
                                                                                        selection_type="NEW_SELECTION", 
                                                                                        where_clause=Expression, 
                                                                                        invert_where_clause="")

In [46]:
print(f"Year {year} has {Count} count!")

Year 2023 has 78 count!


In [8]:
arcpy.env.overwriteOutput = True

In [44]:
# search for duplicated records based on the same values for all fields
start = timeit.default_timer()
result = arcpy.management.FindIdentical(lyr, path+'\\duplicate_incidents.dbf', field_names)
stop = timeit.default_timer()
total_time = stop - start
print(total_time)

145.9570421999997


In [47]:
out_records = []   
for row in arcpy.SearchCursor(result.getOutput(0), fields="IN_FID; FEAT_SEQ"):
    out_records.append([row.IN_FID, row.FEAT_SEQ])

In [48]:
out_records.sort(key = itemgetter(1))

In [49]:
identicals_iter = groupby(out_records, itemgetter(1))

In [50]:
identical_groups = [[item[0] for item in data] for (key, data) in identicals_iter]

In [51]:
len([item for item in identical_groups if len(item) == 1])

1429

In [52]:
len(identical_groups)

1429

In [53]:
[item for item in identical_groups if len(item) > 1]

[]

In [26]:
start = timeit.default_timer()
res = arcpy.management.DeleteIdentical(lyr, field_names)
stop = timeit.default_timer()
total_time = stop - start
print(total_time)

125.17131449999988


In [23]:
lyr

'https://services5.arcgis.com/9s1YtFmLS0YTl10F/arcgis/rest/services/FERNS_for_McKenzie_Catchments/FeatureServer/0'

In [24]:
Expression=f"YEAR <> '{year}'"
selected, count = arcpy.management.SelectLayerByAttribute(in_layer_or_view=lyr, 
                                                          selection_type="NEW_SELECTION",
                                                          where_clause=Expression, 
                                                          invert_where_clause="")

In [26]:
arcpy.env.workspace = wrkspace

In [27]:
arcpy.management.CopyFeatures(selected, f'FERNS_before_{year}')

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\EPA319_NPS_grant\\ForestApplication\\ODF_FACTS_DB\\FERNS\\FERNS_Model\\FERNS_Workspace.gdb\\FERNS_before_2023'>

In [28]:
FERNSSummary_McKenzie_Joined = path + "\\FERNS_Final_Products.gdb\\FERNSSummary_McKenzie_Joined"

In [29]:
Joined = arcpy.management.Append(inputs=[FERNSSummary_McKenzie_Joined],
                                 target=f'FERNS_before_{year}',
                                 schema_type="TEST", 
                                 field_mapping="", 
                                 subtype="", 
                                 expression="")[0]